In [1]:
import os
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from wfdb import rdrecord
from wfdb import processing
aftdb_path = "/aii_data/ecg_waveforms/physionet_original_data/aftdb/learning-set/"
test_a= "/aii_data/ecg_waveforms/physionet_original_data/aftdb/test-set-a/"
test_b="/aii_data/ecg_waveforms/physionet_original_data/aftdb/test-set-b/"
torch.set_num_threads(4)

In [2]:
data=None
for file in ['04015.npz', '04043.npz', '04048.npz', '04126.npz', '04746.npz', '04908.npz', '04936.npz', '05121.npz', '05261.npz',
 '06426.npz', '06453.npz', '06995.npz', '07162.npz', '07879.npz', '07910.npz', '08215.npz', '08434.npz', '08455.npz']:
    info=np.load('/aii/tianq/data/mitafdb16/'+file)
    if type(data)==type(None):
        data=info['data']
        labels=info['label']
    else:
        data=np.append(data, info['data'], axis=0)
        labels=np.append(labels,info['label'] , axis=0)
x= np.append(data[:, :,0], data[:,:,1], axis=0)
y= np.append(labels, labels, axis=0)

info=np.load('/aii/tianq/data/mitafdb16/Challenge_2017.npz') 
x= np.append(x, info['data'], axis=0)
y= np.append(y, info['label'], axis=0)

In [3]:
data=None
for file in ['08405.npz', '08378.npz', '07859.npz', '05091.npz', '08219.npz']:
    info=np.load('/aii/tianq/data/mitafdb16/'+file)
    if type(data)==type(None):
        data=info['data']
        labels=info['label']
    else:
        data=np.append(data, info['data'], axis=0)
        labels=np.append(labels,info['label'] , axis=0)
            
test_x= np.append(data[:, :,0], data[:,:,1], axis=0)
test_y= np.append(labels, labels, axis=0)

In [4]:
data=None
for file in ["n01", "n02" , "n03" , "n04" , "n05", "n06", "n07", "n08", "n09", "n10","s01", "s02" , "s03" , "s04" , "s05", "s06", "s07", "s08", "s09", "s10", "t01", "t02" , "t03" , "t04" , "t05", "t06", "t07", "t08", "t09", "t10"]:
    record = rdrecord(aftdb_path+file)
    if type(data)==type(None):
        data=processing.resample_sig(record.p_signal[:,0].ravel(), 128, 250)
        data=np.append(data, processing.resample_sig(record.p_signal[:,1].ravel(), 128, 250), axis=0)
    else:
        data=np.append(data, processing.resample_sig(record.p_signal[:,0].ravel(), 128, 250), axis=0)
        data=np.append(data, processing.resample_sig(record.p_signal[:,1].ravel(), 128, 250), axis=0)

In [5]:
data1=None
for i in range(data.shape[0]):
    trial=np.split(data[i, :], [4000, 8000, 12000, 15000])
    trial[0]=trial[0].reshape((4000, 1))
    trial[1]=trial[1].reshape((4000, 1))
    trial[2]=trial[2].reshape((4000, 1))
    trial[3]=np.append(trial[3], np.array([0]*1000), axis=0).reshape((4000, 1))
    if type(data1)==type(None):
        data1= np.concatenate((trial[0], trial[2], trial[3], trial[1]), axis=-1)
    else:
        data1=np.concatenate((data1, trial[0], trial[2], trial[3], trial[1]), axis=-1)

In [6]:
x= np.append(x, data1.T, axis=0)
y= np.append(y, np.array([1]*480), axis=0)

In [7]:
data=None
for i in range(1, 31):
    if i<10:
        record = rdrecord(test_a+'a'+str(0)+str(i))
    else:
        record = rdrecord(test_a+'a'+str(i))
    if type(data)==type(None):
        data=processing.resample_sig(record.p_signal[:,0].ravel(), 128, 250)
        data=np.append(data, processing.resample_sig(record.p_signal[:,1].ravel(), 128, 250), axis=0)
    else:
        data=np.append(data, processing.resample_sig(record.p_signal[:,0].ravel(), 128, 250), axis=0)
        data=np.append(data, processing.resample_sig(record.p_signal[:,1].ravel(), 128, 250), axis=0) 
for i in range(1,21):
    if i<10:
        record = rdrecord(test_b+'b'+str(0)+str(i))
    else:
        record = rdrecord(test_b+'b'+str(i))
    data=np.append(data, processing.resample_sig(record.p_signal[:,0].ravel(), 128, 250), axis=0)
    data=np.append(data, processing.resample_sig(record.p_signal[:,1].ravel(), 128, 250), axis=0)

In [8]:
data1=None
for i in range(data.shape[0]):
    trial=np.split(data[i, :], [4000, 8000, 12000, 15000])
    trial[0]=trial[0].reshape((4000, 1))
    trial[1]=trial[1].reshape((4000, 1))
    trial[2]=trial[2].reshape((4000, 1))
    trial[3]=np.append(trial[3], np.array([0]*1000), axis=0).reshape((4000, 1))
    if type(data1)==type(None):
        data1= np.concatenate((trial[0], trial[2], trial[3], trial[1]), axis=-1)
    else:
        data1=np.concatenate((data1, trial[0], trial[2], trial[3], trial[1]), axis=-1)
test_data=data1.T
test_labels=np.array([1]*800)

In [9]:
test_data= np.append(test_data, test_x, axis=0)
test_labels= np.append(test_labels, test_y, axis=0)

In [10]:
np.save( 'training_data',x)
np.save( 'training_labels', y)
np.save( 'test_data',test_data)
np.save( 'test_labels',test_labels)

In [11]:
from scipy.signal import butter, lfilter, freqz, filtfilt, detrend

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=1):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)        
    y = filtfilt(b, a, data) #use filtfilt instead of lfilt to avoid a phase delay induced by the filtering
    return y


def standardlize(in_seq, mean=0, std=1, axis=1):
    in_mean = np.mean(in_seq, axis=axis).reshape(-1,1)
    in_std = np.std(in_seq, axis=axis).reshape(-1,1)
    out_seq =  mean + (in_seq - in_mean ) * std / in_std
    return out_seq

x=butter_bandpass_filter(x, 1, 30, 250, order=1)
x=standardlize(detrend(x))
test_data=butter_bandpass_filter(test_data, 1, 30, 250, order=1)
test_data=standardlize(detrend(test_data))

In [2]:
x= np.load( 'training_data.npy')
y=np.load( 'training_labels.npy')
test_data=np.load( 'test_data.npy')
test_labels=np.load( 'test_labels.npy')

In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
test_data = TensorDataset(torch.from_numpy(test_data), torch.from_numpy(test_labels))
# dataloaders
batch_size = 64
# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, num_workers=4)
ngpu=1
device = torch.device("cuda:2" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
import torch.nn as nn

class SentimentGRU(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, output_size, input_dim, hidden_dim, n_layers, drop_prob=0.15):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.GRU(input_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(0.15)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(1)

        hidden=self.init_hidden(batch_size)
        #hidden = tuple([each.data for each in hidden])
        lstm_out, hidden = self.lstm(x, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        else:
            hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        
        return hidden

In [5]:
# Instantiate the model w/ hyperparams
output_size = 1
input_dim = 1
hidden_dim = 256
n_layers = 2
ngpu=1
#device="cpu"
device = torch.device("cuda:3" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
net = SentimentGRU(output_size, input_dim, hidden_dim, n_layers)
#net.load_state_dict(torch.load("gru6",map_location="cuda:2"))
net.to(device)
print(net)

SentimentGRU(
  (lstm): GRU(1, 256, num_layers=2, dropout=0.15)
  (dropout): Dropout(p=0.15)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [6]:
epsilon=1e-20
def accuracy(outputs, labels):
    temp=(outputs==labels)
    return temp.sum()/ (labels.shape[0])

def true_positive(outputs, labels):
    temp=outputs+labels
    return (temp==2).sum()
    
def true_negative(outputs, labels):
    temp=outputs+labels
    return (temp==0).sum()

def false_positive(outputs, labels):
    temp= labels-outputs
    return (temp<0).sum()

def false_negative(outputs, labels):
    temp=outputs-labels
    return (temp<0).sum()

def precision(outputs, labels):
    return true_positive(outputs, labels)/(true_positive(outputs, labels)+false_positive(outputs, labels))

def recall(outputs, labels):
    return true_positive(outputs, labels)/(true_positive(outputs, labels)+false_negative(outputs, labels))

def BCR(outputs, labels):
    return (precision(outputs, labels)+recall(outputs, labels))/2

In [7]:
def predict(net, test_seq, sequence_length):
    
    net.eval()

    # pad tokenized sequence
    seq_length=sequence_length
    
    # convert to tensor to pass into your model
    #feature_tensor = torch.from_numpy(test_seq)
    feature_tensor=test_seq.view(4000,sequence_length, 1)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    h = tuple([each.data for each in h])
    
    
    # get the output from the model
    feature_tensor = feature_tensor.type(torch.FloatTensor)
    feature_tensor = feature_tensor.to(device)
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
   # print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
  #  if(pred.item()==1):
  #      print("AFIB wave")
 #   else:
   #     print("Normal wave")
    
    return pred.detach().cpu().numpy()

In [ ]:
plot_train=[];plot_val=[]
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_on_gpu=True

# training params

epochs = 50 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 1000
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.to(device)

net.train()

# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    #h = net.init_hidden(batch_size)

    # batch loop
    train_loss=[]
    h=None
    for inputs, labels in train_loader:
        
        counter += 1
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        #h = tuple([each.data for each in h])
        
        bs= inputs.shape[0]
        inputs=inputs.view(4000, bs, 1)
        inputs = inputs.type(torch.FloatTensor)
        #if(train_on_gpu):
        inputs, labels = inputs.to(device), labels.to(device)
        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        train_loss.append(loss.item())

    pred=np.array([]); true_l=np.array([])
    net.eval()
    for inputs, labels in test_loader:
        pred=np.append(pred,predict(net, inputs, inputs.shape[0]), axis=0)
        true_l=np.append(true_l,labels.detach().cpu().numpy(), axis=0)
    test_acc= accuracy(pred, true_l)
    test_prec= precision(pred, true_l)
    test_recall= recall(pred, true_l)
    test_f1= 2*test_prec*test_recall/(test_prec+test_recall)
    print("Accuracy="+ str(test_acc)+"....."+"Precision="+ str(test_prec)+'.....'+"Recall="+ str(test_recall)+'....'+"F1 score="+ str(test_f1))
    net.train()
    print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(np.mean(train_loss)))

    torch.save(net.state_dict(), "lstm"+ str(e))
    plot_train.append(np.mean(train_loss))
    plot_val.append(test_f1)
np.save('training_g_lstm', plot_train)
np.save('Validation_g_lstm', plot_val)

Accuracy=0.6686036789297659.....Precision=0.6472434355058061.....Recall=0.5759635077266804....F1 score=0.6095266243042214
Epoch: 1/50... Step: 1418... Loss: 0.569514...
Accuracy=0.6025501672240803.....Precision=0.5403199477636305.....Recall=0.7703407186743623....F1 score=0.6351460260198795
Epoch: 2/50... Step: 2836... Loss: 0.418934...
Accuracy=0.6650501672240803.....Precision=0.5978915662650602.....Recall=0.7761124557810464....F1 score=0.6754435712549623
Epoch: 3/50... Step: 4254... Loss: 0.341608...
Accuracy=0.6699414715719063.....Precision=0.6093906093906094.....Recall=0.738223794451685....F1 score=0.6676489160176805
Epoch: 4/50... Step: 5672... Loss: 0.294970...
Accuracy=0.6705685618729097.....Precision=0.6056244464127547.....Recall=0.763824241295848....F1 score=0.6755866611774393
Epoch: 5/50... Step: 7090... Loss: 0.266217...
Accuracy=0.6905100334448161.....Precision=0.6194462330972311.....Recall=0.8059951591882331....F1 score=0.7005137748290787
Epoch: 6/50... Step: 8508... Loss: 

In [ ]:
train_on_gpu=True
pred=np.array([]); true_l=np.array([])
for test_data, test_label in test_loader:
    pred=np.append(pred,predict(net, test_data, test_data.shape[0]), axis=0)
    true_l=np.append(true_l,test_label.detach().cpu().numpy(), axis=0)

In [ ]:
test_acc= accuracy(pred, true_l)
test_prec= precision(pred, true_l)
test_recall= recall(pred, true_l)
test_f1= 2*test_prec*test_recall/(test_prec+test_recall)
print("Accuracy="+ str(test_acc))
print("Precision="+ str(test_prec))
print("Recall="+ str(test_recall))
print("F1 score="+ str(test_f1))

In [10]:
plot_val

[0.6095266243042214,
 0.6351460260198795,
 0.6754435712549623,
 0.6676489160176805,
 0.6755866611774393,
 0.7005137748290787,
 0.7109096544242989,
 0.6994780697738303,
 0.6854420859100786,
 0.694111162928732,
 0.6946507552365851,
 0.6895249802440951,
 0.6890177442560055,
 0.6960404380791911,
 0.7026427962489343,
 0.686778141699707,
 0.7024176609911964,
 0.7049047893825736,
 0.7095755875131532,
 0.696213278994736,
 0.7154846141188087,
 0.6933794736362808,
 0.7038718519787562,
 0.6966942148760331,
 0.6962174940898346,
 0.6979332273449921,
 0.6987087517934003,
 0.7010811503781889,
 0.6874889556458738,
 0.7022357282396353,
 0.7095808383233533,
 0.6948083454633673,
 0.6983822439279642,
 0.7097885932837389,
 0.7157956301959311,
 0.708353648626686,
 0.6934448386551783,
 0.6994127516778523,
 0.7117698488696449,
 0.6989070090215129,
 0.7040059347181009,
 0.700677045150718,
 0.7021267943556585,
 0.6958736187845305,
 0.7040132323496125,
 0.6867235184624042,
 0.7056119677405874,
 0.703466495877031